The below is lightly adapted from [Sasank Chilamkurthy](https://chsasank.github.io)'s [transfer learning tutorial for PyTorch](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html), which is distributed under a BSD License.

## Basic Idea for this Notebook
- We did the retraining in the first notebook, `train_model`, and now we will rest it on some example images. 
- We make a "comparison" between the retrained model and the original.
- We do this just to be complete in terms of this "demo", even if it is a toy example.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import os
%matplotlib inline

from tlt_utilities import evaluate_model

Load Data
---------

We load the data again.

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Here we make our notebook GPU / CPU agnostic
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Check for GPU

Again, we do this for the user's benefit.

In [ ]:
# gtm:ignore
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU unavailable. Using CPU.")

### Load retrained model and classify some images

In [ ]:
retrained_model = torch.load('../models/ants-bees.pt', map_location=device)

In [ ]:
evaluate_model(retrained_model, dataloaders, class_names, num_images=10)

### Now let's try some with the original model.

In [ ]:
original_model = torch.load('../models/original.pt', map_location=device)

In [ ]:
evaluate_model(original_model, dataloaders, class_names, num_images=10)